# Shared Task: Critical thinking generation
## Loadind Data


In [ ]:
pip install -q social_agents/

In [ ]:
pip show social_agents

In [ ]:
import social_agents

In [15]:
import tqdm
# Reading the data
import json
import pandas as pd

from social_agents.utils import get_st_data


for key, line in tqdm.tqdm(get_st_data("sample").items()):
    print(key)

    print(line['intervention'])
    input_text = line['intervention']
    break


  0%|          | 0/5 [00:00<?, ?it/s]

CLINTON_1_1
CLINTON: "The central question in this election is really what kind of country we want to be and what kind of future we 'll build together
Today is my granddaughter 's second birthday
I think about this a lot
we have to build an economy that works for everyone , not just those at the top
we need new jobs , good jobs , with rising incomes
I want us to invest in you
I want us to invest in your future
jobs in infrastructure , in advanced manufacturing , innovation and technology , clean , renewable energy , and small business
most of the new jobs will come from small business
We also have to make the economy fairer
That starts with raising the national minimum wage and also guarantee , finally , equal pay for women 's work
I also want to see more companies do profit-sharing"


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [ ]:
from IPython.display import Image, display
import os, getpass

## Prerequisites
env, LLM, ...

In [ ]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("MISTRAL_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "shared_task_critical_questions"


In [ ]:
# temperature = 0 not for o3-mini


##  1. Zero Shot LLM to start with o3-mini

In [ ]:

from social_agents.graph_tools import BasicCQModel
model_name = "o3-mini"

basic_agent = BasicCQModel(llm_name = model_name, temperature=None ) #interrupt_before=['human_feedback'], checkpointer=memory)


display(Image(basic_agent.graph.get_graph(xray=1).draw_mermaid_png()))

# RUN
# basic_agent.run_experiment()



### Evaulate 

- Run shell 

```shell

python3 eval_scripts/evaluation.py \
    --metric similarity \
    --input_path data_splits/validation.json \
    --submission_path output/output_o3-mini_temperatureNA.json \
    --threshold 0.6 

```

- OUTPUT : output/output_o3-mini_temperatureNA._eval_similarity_06.json



**Overall count**

| **Questions Labels** |  **#**  | **ratio** |
|:--------------------|-------:|---------:|
| useful               | **329** |  **0,59** |
| unhelpful            |      63 |      0,11 |
| Invalid              |       8 |      0,01 |
| Not able to evaluate |     158 |      0,28 |
| **Total**            | **558** |         1 |


**Overall count within each argument**

| **n/3 useful questions per arg** | **# of arguments** | **ratio** |
|:-------------------------------:|-------------------:|:---------:|
|                               0/3 |                 17 |      0,10 |
|                               1/3 |                 51 |      0,27 |
|                               2/3 |             **76** |      0,40 |
|                               3/3 |                 42 |      0,23 |
| **Total**                       |            **186** |         1 |

## Social Agents

In [16]:
import dataclasses
from typing import Dict, Literal, TypedDict

from social_agents.graph_tools import CQSTAbstractAgent
from langgraph.graph import START, END, StateGraph
from social_agents.objects import  CriticalQuestionList
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
class Confirmation(BaseModel):
    confirmation: str = Field(
        description="The LLM confirms the role assigned to it by simply saying ok."
    )


class SocialAgentAnswer(BaseModel):
    critical_question_list: CriticalQuestionList = Field(
        description="The list of all the critical questions and their ranks to criticize and reveal the weaknesses of an argument."
    )
    question_type: Literal["debate", "reflect", "question"]
    prompt: str


class SocialAgentState(TypedDict):
    input_arg: str
    current_round: int = -1  # 1:start
    collaborative_strategy: list

    round_answer_dict: dict[str, list[SocialAgentAnswer]] = dataclasses.field(
        default_factory=dict
    )  # key is agent and value array of N CriticalQuestionList (based on number of rounds)

    # OutPut
    final_cq: CriticalQuestionList


# @dataclasses.dataclass
# class SocialAgentsModel(CQSTAbstractAgent):
collaborative_strategy = ["debate", "reflect", "debate"]
agent_trait_lst: list = ["easy_going", "overconfident", "easy_going"]
llm_name = "o3-mini-2025-01-31"
temperature = 0.0
llm_lst = [
    CQSTAbstractAgent._init_llm(llm_name, temperature)
    for _ in range(len(agent_trait_lst))
]
validator_llm = CQSTAbstractAgent._init_llm(llm_name, temperature)


def llm_role_node(state: SocialAgentState):
    are_all_roles_confirmed = True
    for i, llm in enumerate(llm_lst):
        structured_llm = llm.with_structured_output(Confirmation)
        with open(f"prompts/trait_{agent_trait_lst[i]}.txt", "r") as f:
            trait_prompt = f.read()

        response = structured_llm.invoke(
            trait_prompt
        )
        if response.confirmation.lower() == "ok":
            print("role confirmed")
            
        else:
            print(f"Unconfirmed Role: {response.confirmation}")
            are_all_roles_confirmed = False
            break
            
    return {"role_confirmed": are_all_roles_confirmed,
            "current_round": -1}

def is_role_confirmed(state):
    if state["role_confirmed"]:
        return "question_node"
    else: return END

def question_node(state: SocialAgentState):
    round_answer_dict = {}
    for i, llm in enumerate(llm_lst):
        structured_llm = llm.with_structured_output(CriticalQuestionList)
        # with open("prompts/system.txt", "r") as f:
        #    system_prompt = f.read()
        with open("prompts/question.txt", "r") as file:
            instructions = file.read()

        instructions = instructions.format(
            input_arg=state["input_arg"],
        )
        response = structured_llm.invoke(instructions)
        answer: SocialAgentAnswer = SocialAgentAnswer(
            critical_question_list=response,
            question_type="question",
            prompt=instructions,
        )
        
        print(1)
        round_answer_dict[f"agent{i}"] = [answer]
        print(2)
        print(3)

    return {"current_round": state["current_round"] + 1,
        "round_answer_dict":round_answer_dict }
    
    
    
    


def debate_node(state: SocialAgentState):
    for i, llm in enumerate(llm_lst):
        structured_llm = llm.with_structured_output(CriticalQuestionList)

        with open("prompts/strategy_debate.txt", "r") as file:
            instructions = file.read()

        # Get others answer
        round_answer_dict = state["round_answer_dict"]
        print(round_answer_dict)
        others_answers = [
            round_answer_dict[f"agent{x}"][-1] for x, _ in enumerate(llm_lst) if x != i
        ]

        other_agents_response_str = ""
        other_cq_str = "\n- critical question {id}: '{cq}', reasoning: '{reason}'.\n"
        for a_num, other_ in enumerate(others_answers):
            other_agents_response_str += f"Agent{a_num+1}:"
            for cq in other_.critical_question_list.critical_questions:
                other_agents_response_str = (
                    other_agents_response_str
                    + other_cq_str.format(id = cq.id, cq=cq.critical_question, reason=cq.reason)
                )
            other_agents_response_str += "\n\n"

        instructions = instructions.format(
            input_arg=state["input_arg"],
            other_agents_response=other_agents_response_str,
        )

        response = structured_llm.invoke(instructions)
        answer: SocialAgentAnswer = SocialAgentAnswer(
            critical_question_list=response,
            question_type="debate",
            prompt=instructions,
        )

        round_answer_dict[f"agent{i}"].append(answer)
        state["round_answer_dict"] = round_answer_dict

    
    return {"current_round": state["current_round"] + 1,
            "round_answer_dict":round_answer_dict }
    
    
    
def decide_next(state) -> Literal["debate", "reflect", "validate"]:
    if len(state["collaborative_strategy"]) > state["current_round"]:
        print(f'MOVING TO STATE {state["collaborative_strategy"][state["current_round"]]}')
        return f'{state["collaborative_strategy"][state["current_round"]]}_node'
    else:
        print("going to validate node")
        return "validate_node"



def reflect_node(state: SocialAgentState):
    for i, llm in enumerate(llm_lst):
        structured_llm = llm.with_structured_output(CriticalQuestionList)

        with open("prompts/strategy_reflect.txt", "r") as file:
            instructions = file.read()

        # Get others answer
        round_answer_dict = state["round_answer_dict"]
        previous_answer = round_answer_dict[f"agent{i}"][-1]
        instructions = instructions.format(
            input_arg=state["input_arg"], previous_answer=previous_answer
        )

        response = structured_llm.invoke(instructions)
        answer: SocialAgentAnswer = SocialAgentAnswer(
            critical_question_list=response,
            question_type="debate",
            prompt=instructions,
        )

        round_answer_dict[f"agent{i}"].append(answer)
        state["round_answer_dict"] = round_answer_dict
    return {"current_round": state["current_round"] + 1,
        "round_answer_dict":round_answer_dict }
    
    


def validate_node(state: SocialAgentState):
    structured_llm = validator_llm.with_structured_output(CriticalQuestionList)
    with open("prompts/validator.txt", "r") as file:
        instructions = file.read()
    # Get others answer
    round_answer_dict = state["round_answer_dict"]
    others_answers = [
        round_answer_dict[f"agent{x}"][-1] for x, _ in enumerate(llm_lst)
    ]
    other_agents_response_str = ""
    other_cq_str = "\n- critical question {id}: '{cq}'.\n" # reasoning: '{reason}'
    for a_num, other_ in enumerate(others_answers):
        other_agents_response_str += f"Agent{a_num+1}:"
        for cq in other_.critical_question_list.critical_questions:
            other_agents_response_str = (
                other_agents_response_str
                + other_cq_str.format(id = cq.id, cq=cq.critical_question, reason=cq.reason)
            )
        other_agents_response_str += "\n\n"
    instructions = instructions.format(
        input_arg=state["input_arg"],
        other_agents_response=other_agents_response_str,
    )
    response = structured_llm.invoke(instructions)
    print(instructions)
    return {"final_cq": response}



builder = StateGraph(SocialAgentState)

builder.add_node("llm_role_node", llm_role_node)
builder.add_node("question_node", question_node)
builder.add_node("debate_node", debate_node)
builder.add_node("reflect_node", reflect_node)
builder.add_node("validate_node", validate_node)

builder.add_edge(START, "llm_role_node")
builder.add_edge("llm_role_node", "question_node")
builder.add_conditional_edges("question_node", decide_next, ["debate_node", "reflect_node", "validate_node"])
builder.add_conditional_edges("debate_node", decide_next, ["debate_node", "reflect_node", "validate_node"])
builder.add_conditional_edges("reflect_node", decide_next, ["debate_node", "reflect_node", "validate_node"])
builder.add_edge("validate_node", END)
from langgraph.checkpoint.memory import MemorySaver

from langchain_core.runnables.graph import CurveStyle
memory = MemorySaver()
graph = builder.compile( checkpointer=memory)


In [ ]:
import uuid

thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}
graph.invoke({"collaborative_strategy": collaborative_strategy, "input_arg": "I am human therefore I am mortal"}, config)

role confirmed
role confirmed
role confirmed
1
2
3
1
2
3


In [ ]:
display(Image(graph.get_graph(xray=True).draw_mermaid_png(curve_style=CurveStyle.LINEAR, padding=0)))